In [1]:
def zip_county_highways(full_fc_path, output_folder, county):
    '''
    This function creates a zip file containing a file geodatabase with 
    a feature class containing only the highways for a specific county.

    Parameters:
        full_fc_path (str): The full path to the input feature class
        output_folder (str): The path to the output folder
        county (str): The name of the county to extract highways for

    Returns:
        str: The path to the zipped file geodatabase
    '''
    
    # remove spaces from county name
    county_no_spaces = county.replace(" ", "_")
    
    # create a file geodatabase
    fgdb = arcpy.management.CreateFileGDB(
        out_folder_path = output_folder,
        out_name = f"{county_no_spaces}_Output"
    )

    # Create a feature class
    output_fc = arcpy.conversion.ExportFeatures(
        in_features = full_fc_path,
        out_features = os.path.join(fgdb[0], 
                                    f"{county_no_spaces}_Highways"),
        where_clause = f"NAMELSAD = '{county}'"
    )
    
    source_fgdb_path = pathlib.Path(fgdb[0])

    # The name of the folder to place the File Geodatabase in
    fgdb_folder_name = source_fgdb_path.stem
    # The location of the folder to place the File Geodatabase in
    fgdb_folder_location = source_fgdb_path.parent
    # The path to the folder to place the File Geodatabase in
    fgdb_folder_path = fgdb_folder_location.joinpath(fgdb_folder_name)
    # The path of our copied File Geodatabase
    fgdb_path = fgdb_folder_path.joinpath(source_fgdb_path.name)

    # Copy the file geodatabase into a temp folder
    shutil.copytree(
        source_fgdb_path, 
        fgdb_path, 
        ignore=shutil.ignore_patterns('*.lock')
    )

    # Zip the File Geodatabase
    zipped_fgdb = shutil.make_archive(
        base_name=fgdb_folder_path,  # The name of the archive, not including the file extension
        format="zip",  # The archive format
        root_dir=fgdb_folder_path,  # The directory to archive
    ) 

    # Delete the file geodatabase
    arcpy.management.Delete(fgdb)

    # Remove the temp folder
    shutil.rmtree(fgdb_folder_path)
    
    # Return the zip file path
    return zipped_fgdb

In [2]:
# import packages
import arcpy
import shutil
import pathlib
import os

In [3]:
# input file geodatabase path
full_fc_path = ".\Chapter 13.gdb\Highways_intersect"

# output folder
output_folder = ".\zipped_outputs"

In [ ]:
# export a zip file for Alameda County
zip_county_highways(full_fc_path, output_folder, "Alameda County")

In [ ]:
if __name__ == "__main__":
    # input file geodatabase path
    full_fc_path = arcpy.GetParameterAsText(0)

    # output folder
    output_folder = arcpy.GetParameterAsText(1)

    # which county to export
    county = arcpy.GetParameterAsText(2)

    # export a zip file for the County
    zip_county_highways(full_fc_path, output_folder, county)

Copyright 2025 Esri